In [10]:
import pandas as pd
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')

In [2]:
df = pd.read_csv('BTC_ETH_LTC_16NOV.csv')

In [12]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

In [13]:
symbols = ['BTC']

In [14]:
nstd = 3

for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])


In [15]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier, symbols):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier # VIP level 0, paying fees with BNB
        self.symbols = symbols
        
        self.bottoms = {}
        self.reset_bottoms() 
        
        self.tops = {}
        self.reset_tops()
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append([self.balance_unit, time, sell_price])
        self.balance_unit = 'USDT'
        
    def reset_bottoms(self):
        for symbol in self.symbols:
            self.bottoms[symbol] = 'none'
        
    def reset_tops(self):
        for symbol in self.symbols:
            self.tops[symbol] = 'none'
        

In [7]:
'''
innitially when price is within channel = none
when it crossed first bottom = hit
when it crosses above bottom after hit = released
'''

'\ninnitially when price is within channel = none\nwhen it crossed first bottom = hit\nwhen it crosses above bottom after hit = released\n'

In [22]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925, symbols=symbols)

for i in range(len(df)):  
    #means looking for something to buy
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if env.bottoms[symbol] == 'hit' and df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                env.bottoms[symbol] = 'released'
            
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 20: #buy signal
                if env.bottoms[symbol] == 'released':
                    env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                    env.reset_bottoms()
                    break
                else:
                    env.bottoms[symbol] = 'hit'
    
    #means looking to sell
    if env.balance_unit != 'USDT':
        if env.tops[env.balance_unit] == 'hit' and df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]:
            env.tops[env.balance_unit] = 'released'
        
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 80: #sell signal
            if env.tops[env.balance_unit] == 'released':
                env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])
                env.reset_tops()
            else:
                env.tops[env.balance_unit] = 'hit'

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])
    
   

In [23]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 4
num sells: 4
ending balance: 96.5748894792675 USDT


In [31]:
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    print('\n')

['BTC', '2022-11-16 10:56:00', 16792.25083691415]
['BTC', '2022-11-16 11:57:00', 16855.338406492894]
P/L = 0.37569454024626303


['BTC', '2022-11-16 14:22:00', 16928.967577869214]
['BTC', '2022-11-16 15:18:00', 16922.039155381153]
P/L = -0.0409264324962022


['BTC', '2022-11-16 16:11:00', 16889.292258865134]
['BTC', '2022-11-16 17:02:00', 16896.161880173106]
P/L = 0.04067441786594905


['BTC', '2022-11-16 18:00:00', 16815.619922646805]
['BTC', '2022-11-16 20:04:00', 16773.04289935849]
P/L = -0.2531992485806267


['BTC', '2022-11-16 20:36:00', 16691.752089485755]
['BTC', '2022-11-16 20:43:00', 16754.65165393053]
P/L = 0.37683020995977756


['BTC', '2022-11-16 21:49:00', 16709.191793541515]
['BTC', '2022-11-16 23:33:00', 16674.15172413135]
P/L = -0.20970535165986126


['BTC', '2022-11-16 23:56:00', 16474.124285860224]
['BTC', '2022-11-17 00:34:00', 16578.37796267964]
P/L = 0.6328328899940061


['BTC', '2022-11-17 01:46:00', 16490.00050053416]
['BTC', '2022-11-17 02:41:00', 16505.97528804